# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Paul Florencio Rojas Quispe  <br>
Url: https://github.com/paul0610/03MIAR---Algoritmos-de-Optimizacion---2023/tree/main/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1zC2OCw32ZyoOp05GlDe3cxgxZZa0hdUE?usp=sharing<br>
<br>
**Problema:** Sesiones de doblaje <br>
<br>
**Descripción del problema:**
<br>
<br>
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. <br> <br>El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.
<br>
<br>
**Los datos son:**
<br>
**Número de actores:** 10<br>
**Número de tomas :** 30<br>
**Actores/Tomas:** https://docs.google.com/spreadsheets/d/1ODVSs3OPrTa36govaoYQbqRC9LhFp9PN/edit?usp=sharing&ouid=100668873241613497583&rtpof=true&sd=true
<br><br>
1 indica que el actor participa en la toma<br>
0 en caso contrario







                                        

## Análisis del Problema
Este problema se enfoca en mejorar la coordinación del proceso de doblaje en películas, con el objetivo de optimizar la agenda de los actores de doblaje para asegurar que las sesiones en las que participan simultáneamente estén bien sincronizadas, a la vez que se busca reducir los costes derivados de su contratación.<br><br>

Estos costes se calculan en función de los días de trabajo en lugar del número de escenas dobladas. Para abordar esta complejidad, se utilizan algoritmos evolutivos y técnicas de optimización que permiten refinar las soluciones a través de mutaciones aleatorias y la selección de las opciones más prometedoras en cada ciclo.<br><br>

Para resolver este problema, se emplean algoritmos y técnicas de optimización, especialmente algoritmos evolutivos, que buscan mejorar continuamente las soluciones mediante mutaciones aleatorias y la selección de las mejores soluciones en cada iteración. Se incorporan además estrategias para verificar y corregir las propuestas, asegurando que se ajusten a las limitaciones del proyecto. La efectividad de las soluciones se mide mediante una función específica que evalúa el coste total basado en los días laborales de los actores.<br><br>

La evaluación de la calidad de las soluciones se lleva a cabo mediante una función objetivo que calcula el costo total de una solución, considerando el número de días en los que trabajan los actores. Además, se implementan métodos de verificación y reparación de soluciones para garantizar que todas las propuestas sean viables y cumplan con las restricciones del problema. El objetivo es encontrar un plan de trabajo para el doblaje que no solo sea viable sino que también optimice los recursos y reduzca los gastos.
<br><br>

El espacio de soluciones para este problema incluye todas las posibles combinaciones de asignación de tomas a días, de tal manera que se cumplan las restricciones de participación de actores y el límite de tomas por día. Cada solución en este espacio es una manera específica de distribuir las 30 tomas a través de un número de días, con hasta 6 tomas por día. Matemáticamente, podrías pensar en este espacio como el conjunto de todas las particiones posibles de las tomas en grupos de hasta 6, donde cada grupo representa las tomas asignadas a un día particular.<br><br>

## Funciones de la Solución

1. `objective_function(solution, schedule_matrix, debug=False)`: Calcula el costo total de una solución dada, que representa el número total de días en los que trabajan los actores.

2. `greedy_initial_solution(schedule_matrix)`: Genera una solución inicial voraz ordenando las tomas de mayor a menor cantidad de actores por día.

3. `swap(solution, day1, day2, take1, take2)`: Intercambia dos tomas entre dos días en una solución dada.

4. `local_search(initial_solution, schedule_matrix)`: Realiza una búsqueda local para mejorar una solución inicial intercambiando tomas entre días.

5. `validate_solution(solution, schedule_matrix)`: Verifica si una solución dada cumple con las restricciones del problema, como que cada toma y cada día sean únicos.

6. `remove_duplicate_takes(solution)`: Elimina tomas duplicadas dentro de la misma solución.

7. `add_missing_takes(solution, schedule_matrix)`: Agrega tomas faltantes a una solución, asegurándose de que todas las tomas estén presentes al menos una vez.

8. `get_possible_solution(solution, schedule_matrix, do_local_search=False)`: Obtiene una solución posible a partir de una solución dada, corrigiendo cualquier inconsistencia y, opcionalmente, aplicando una búsqueda local.

9. `evolutionary_algorithm(initial_solution, population_size, num_epochs, schedule_matrix)`: Implementa un algoritmo evolutivo para buscar soluciones óptimas, generando una población de soluciones y aplicando mutaciones para mejorarlas a lo largo de varias épocas.

10. `multi_start_evolutionary_algorithm(population_size, num_epochs, num_starts, schedule_matrix)`: Aplica el algoritmo evolutivo múltiples veces desde diferentes soluciones iniciales, buscando la mejor solución encontrada entre todas.


In [1]:
import numpy as np
from tabulate import tabulate

# Definir los datos iniciales
datos_iniciales = [
    [1,1,1,1,1,0,0,0,0,0],
    [0,0,1,1,1,0,0,0,0,0],
    [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0],
    [0,1,0,1,0,0,0,1,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0],
    [1,1,1,0,1,0,0,1,0,0],
    [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0],
    [1,0,1,0,0,1,0,0,0,0],
    [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [1,0,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1],
    [1,0,1,0,1,0,0,0,1,0],
    [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0],
    [1,0,0,0,1,1,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0]
]

# Definir los nombres de los actores
nombres_actores = ["Actor 1", "Actor 2", "Actor 3", "Actor 4", "Actor 5",
                   "Actor 6", "Actor 7", "Actor 8", "Actor 9", "Actor 10"]

# Definir los nombres de las tomas
nombres_tomas = ["Toma {}".format(i+1) for i in range(len(datos_iniciales))]

# Agregar los nombres de los actores como primer elemento de cada fila de datos
datos_con_nombres = [[nombre] + fila for nombre, fila in zip(nombres_tomas, datos_iniciales)]

# Imprimir la tabla usando tabulate con formato Markdown
tabla_markdown = tabulate(datos_con_nombres, headers=["Toma"] + nombres_actores, tablefmt="pipe")
print(tabla_markdown)

# Convertir a un array de numpy para calcular las sumas
datos_np = np.array(datos_iniciales)

# Calcular la suma de las filas y las columnas
suma_filas = np.sum(datos_np, axis=1)
suma_columnas = np.sum(datos_np, axis=0)

# Mostrar las sumas como arrays
print("\nSuma de las columnas:")
print(suma_columnas)
print("\nSuma de las filas:")
print(suma_filas)



| Toma    |   Actor 1 |   Actor 2 |   Actor 3 |   Actor 4 |   Actor 5 |   Actor 6 |   Actor 7 |   Actor 8 |   Actor 9 |   Actor 10 |
|:--------|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|-----------:|
| Toma 1  |         1 |         1 |         1 |         1 |         1 |         0 |         0 |         0 |         0 |          0 |
| Toma 2  |         0 |         0 |         1 |         1 |         1 |         0 |         0 |         0 |         0 |          0 |
| Toma 3  |         0 |         1 |         0 |         0 |         1 |         0 |         1 |         0 |         0 |          0 |
| Toma 4  |         1 |         1 |         0 |         0 |         0 |         0 |         1 |         1 |         0 |          0 |
| Toma 5  |         0 |         1 |         0 |         1 |         0 |         0 |         0 |         1 |         0 |          0 |
| Toma 6  |         1 |         1 |         0 |         1 |         1

In [14]:
import numpy as np

# Definimos la función objetivo
def objective_function(solution, schedule_matrix, debug=False):
    total_working_days = 0
    for day, takes_day in solution.items():
        # Creamos un conjunto de los actores que trabajaron este día.
        actors_day = set()
        for take in takes_day:
            # Utilizamos la función np.where para filtrar los actores que trabajan ese día.
            # Al comparar schedule_matrix[take-1]==1, obtenemos una lista de booleanos.
            # Pasamos esta lista de booleanos a np.where, que nos retorna los índices de los valores True.
            # Luego obtenemos el índice 0 ya que retorna una matriz, y sumamos 1 porque los índices comienzan en cero.
            actors_take = set(np.where(schedule_matrix[take-1] == 1)[0] + 1)
            # Usamos la función update, que puede recibir un iterable y los agrega a un conjunto.
            actors_day.update(actors_take)

        if debug:
            print("Tomas del", str(day) + ":", takes_day)
            print("Actores del", str(day) + ":", actors_day)

        total_working_days += len(actors_day)
    return total_working_days

# Datos de prueba
tabla_np = np.array([
    [1,1,1,1,1,0,0,0,0,0],
    [0,0,1,1,1,0,0,0,0,0],
    [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0],
    [0,1,0,1,0,0,0,1,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0],
    [1,1,1,0,1,0,0,1,0,0],
    [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0],
    [1,0,1,0,0,1,0,0,0,0],
    [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [1,0,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1],
    [1,0,1,0,1,0,0,0,1,0],
    [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0],
    [1,0,0,0,1,1,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0]
])

# Prueba de la función objetivo
initial_solution = {
    "dia 1" : [1, 2, 3, 4, 5, 6],
    "dia 2" : [7, 8, 9, 10, 11, 12],
    "dia 3" : [13, 14, 15, 16, 17, 18],
    "dia 4" : [19, 20, 21, 22, 23, 24],
    "dia 5" : [25, 26, 27, 28, 29, 30]
}

print("Total de días trabajados: ", objective_function(initial_solution, tabla_np, True))


Tomas del dia 1: [1, 2, 3, 4, 5, 6]
Actores del dia 1: {1, 2, 3, 4, 5, 7, 8}
Tomas del dia 2: [7, 8, 9, 10, 11, 12]
Actores del dia 2: {1, 2, 3, 4, 5, 6, 8, 9}
Tomas del dia 3: [13, 14, 15, 16, 17, 18]
Actores del dia 3: {1, 2, 3, 4, 5, 6, 7, 10}
Tomas del dia 4: [19, 20, 21, 22, 23, 24]
Actores del dia 4: {1, 2, 3, 4, 5, 6, 8}
Tomas del dia 5: [25, 26, 27, 28, 29, 30]
Actores del dia 5: {1, 2, 3, 4, 5, 6, 9, 10}
Total de días trabajados:  38


In [3]:
def greedy_initial_solution(schedule_matrix):
    num_takes = len(schedule_matrix)

    # Calculamos la cantidad de actores para cada toma
    num_actors_per_take = [np.sum(take) for take in schedule_matrix]

    # Ordenamos las tomas de forma descendente según la cantidad de actores
    sorted_takes = sorted(range(num_takes), key=lambda i: num_actors_per_take[i], reverse=True)

    print(sorted_takes)

    solution = {}
    day = 1
    takes_per_day = 0
    for take in sorted_takes:
        if takes_per_day == 6:  # Si ya se asignaron 6 tomas a este día, pasamos al siguiente día
            day += 1
            takes_per_day = 0
        if solution.get("dia " + str(day)) != None:
            solution["dia " + str(day)].append(take + 1)
        else:
            solution["dia " + str(day)] = [take + 1]
        takes_per_day += 1

    return solution

greedy_solution = greedy_initial_solution(tabla_np)

# Imprimir cada día en una línea diferente
for day, takes in greedy_solution.items():
    print(day + ":", takes)

print(objective_function(greedy_solution, tabla_np, True))


[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]
dia 1: [1, 11, 12, 4, 6, 7]
dia 2: [10, 20, 22, 25, 26, 2]
dia 3: [3, 5, 8, 9, 13, 14]
dia 4: [15, 29, 16, 17, 18, 19]
dia 5: [21, 23, 24, 27, 28, 30]
Tomas del dia 1: [1, 11, 12, 4, 6, 7]
Actores del dia 1: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 2: [10, 20, 22, 25, 26, 2]
Actores del dia 2: {1, 2, 3, 4, 5, 6, 9, 10}
Tomas del dia 3: [3, 5, 8, 9, 13, 14]
Actores del dia 3: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 4: [15, 29, 16, 17, 18, 19]
Actores del dia 4: {1, 2, 3, 4, 5, 6, 7, 10}
Tomas del dia 5: [21, 23, 24, 27, 28, 30]
Actores del dia 5: {1, 3, 4, 5, 6, 8}
38


In [4]:
import copy

# Definimos una función swap que intercambia dos posiciones
def swap(solution, day1, day2, take1, take2):
    # Hacemos una copia profunda de la solución
    new_solution = copy.deepcopy(solution)

    # Verificamos si los días especificados existen en el diccionario
    if "dia" + str(day1) in new_solution and "dia" + str(day2) in new_solution:
        # Verificamos si las posiciones especificadas existen en los días correspondientes
        if take1 < len(new_solution["dia" + str(day1)]) and take2 < len(new_solution["dia" + str(day2)]):
            # Intercambiamos las tomas
            new_solution["dia" + str(day1)][take1], new_solution["dia" + str(day2)][take2] = \
                new_solution["dia" + str(day2)][take2], new_solution["dia" + str(day1)][take1]
        else:
            print("Error: Las posiciones especificadas no existen en los días correspondientes.")
    else:
        print("Error: Los días especificados no existen en la solución.")

    return new_solution

# Probamos la función swap
new_solution = swap(greedy_solution, 1, 2, 0, 0)
if new_solution:
    print(objective_function(new_solution, tabla_np, True))


Error: Los días especificados no existen en la solución.
Tomas del dia 1: [1, 11, 12, 4, 6, 7]
Actores del dia 1: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 2: [10, 20, 22, 25, 26, 2]
Actores del dia 2: {1, 2, 3, 4, 5, 6, 9, 10}
Tomas del dia 3: [3, 5, 8, 9, 13, 14]
Actores del dia 3: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 4: [15, 29, 16, 17, 18, 19]
Actores del dia 4: {1, 2, 3, 4, 5, 6, 7, 10}
Tomas del dia 5: [21, 23, 24, 27, 28, 30]
Actores del dia 5: {1, 3, 4, 5, 6, 8}
38


In [5]:
import random

def local_search(initial_solution, schedule_matrix):
    best_solution = initial_solution
    best_cost = objective_function(best_solution, schedule_matrix)

    num_days = len(initial_solution)
    if num_days < 2:
        print("Error: La solución inicial debe contener al menos dos días.")
        return None

    for i in range(50):
        day1, day2 = random.sample(range(1, num_days + 1), 2)  # Seleccionamos dos días diferentes al azar

        # Verificamos si las claves de los días existen en el diccionario initial_solution
        if "day"+str(day1) in initial_solution and "day"+str(day2) in initial_solution:
            take1 = random.randint(0, len(initial_solution["day"+str(day1)]) - 1)
            take2 = random.randint(0, len(initial_solution["day"+str(day2)]) - 1)
        else:
            print("Error: Las claves de los días no existen en la solución inicial.")
            return None

        new_solution = swap(best_solution, day1, day2, take1, take2)
        cost = objective_function(new_solution, schedule_matrix)

        if cost < best_cost:
            best_solution = new_solution
            best_cost = cost

    return best_solution

initial_solution = greedy_initial_solution(tabla_np)
print(objective_function(initial_solution, tabla_np, True))

local_search_result = local_search(initial_solution, tabla_np)
if local_search_result:
    print(objective_function(local_search_result, tabla_np, True))


[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]
Tomas del dia 1: [1, 11, 12, 4, 6, 7]
Actores del dia 1: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 2: [10, 20, 22, 25, 26, 2]
Actores del dia 2: {1, 2, 3, 4, 5, 6, 9, 10}
Tomas del dia 3: [3, 5, 8, 9, 13, 14]
Actores del dia 3: {1, 2, 3, 4, 5, 6, 7, 8}
Tomas del dia 4: [15, 29, 16, 17, 18, 19]
Actores del dia 4: {1, 2, 3, 4, 5, 6, 7, 10}
Tomas del dia 5: [21, 23, 24, 27, 28, 30]
Actores del dia 5: {1, 3, 4, 5, 6, 8}
38
Error: Las claves de los días no existen en la solución inicial.


In [7]:
def validate_solution(solution, schedule_matrix):
    unique_takes = set()

    for day, takes in solution.items():
        unique_takes_day = set(takes)

        if len(unique_takes_day) != len(takes):
            return False

        if len(unique_takes.intersection(unique_takes_day)) > 0:
            return False

        unique_takes.update(unique_takes_day)

    if len(unique_takes) != len(schedule_matrix):
        return False

    return True

# Verificamos la solución inicial voraz
greedy_initial_solution = greedy_initial_solution(tabla_np)
print(validate_solution(greedy_initial_solution, tabla_np))

# Verificamos una solución no válida
invalid_solution = {
    "dia1": [1, 1, 3, 4, 5, 6],
    "dia2": [7, 8, 9, 10, 11, 12],
    "dia3": [13, 14, 15, 16, 17, 18],
    "dia4": [19, 20, 21, 22, 23, 24],
    "dia5": [25, 26, 27, 28, 29, 30]
}

print(validate_solution(invalid_solution, tabla_np))


[0, 10, 11, 3, 5, 6, 9, 19, 21, 24, 25, 1, 2, 4, 7, 8, 12, 13, 14, 28, 15, 16, 17, 18, 20, 22, 23, 26, 27, 29]
True
False


In [8]:
def remove_duplicate_takes(solution):
    used_takes = set()
    for day, takes in solution.items():
        for i in range(len(takes)):
            if takes[i] in used_takes:
                solution[day][i] = 0
            else:
                used_takes.add(takes[i])
    return solution

# Ejemplo de uso
initial_solution_invalid = {
    "day1": [1, 1, 3, 4, 5, 6],
    "day2": [7, 8, 3, 3, 11, 12],
    "day3": [13, 1, 15, 16, 17, 18],
    "day4": [19, 20, 1, 22, 23, 24],
    "day5": [25, 5, 27, 28, 5, 30]
}

cleaned_solution = remove_duplicate_takes(copy.deepcopy(initial_solution_invalid))
display(cleaned_solution)

{'day1': [1, 0, 3, 4, 5, 6],
 'day2': [7, 8, 0, 0, 11, 12],
 'day3': [13, 0, 15, 16, 17, 18],
 'day4': [19, 20, 0, 22, 23, 24],
 'day5': [25, 0, 27, 28, 0, 30]}

In [9]:
def add_missing_takes(solution, schedule_matrix):
    used_takes = set()
    for day, takes in solution.items():
        for take in takes:
            if take != 0:  # Ignorar los ceros
                used_takes.add(take)

    missing_takes = list(set(range(1, len(schedule_matrix) + 1)) - used_takes)
    random.shuffle(missing_takes)

    for day, takes in solution.items():
        for i in range(len(takes)):
            if takes[i] == 0 and missing_takes:
                new_take = missing_takes.pop(0)
                while new_take == 0 or new_take in takes:
                    new_take = missing_takes.pop(0)
                solution[day][i] = new_take

    return solution

# Ejemplo de uso
invalid_solution = {
    "dia1": [1, 1, 3, 4, 5, 6],
    "dia2": [7, 8, 3, 3, 11, 12],
    "dia3": [13, 1, 15, 16, 17, 18],
    "dia4": [19, 20, 1, 22, 23, 24],
    "dia5": [25, 5, 27, 28, 5, 30]
}

pre_repaired_solution = remove_duplicate_takes(copy.deepcopy(invalid_solution))
repaired_solution = add_missing_takes(pre_repaired_solution, tabla_np)

display(repaired_solution)

print(validate_solution(repaired_solution, tabla_np))

{'dia1': [1, 29, 3, 4, 5, 6],
 'dia2': [7, 8, 26, 9, 11, 12],
 'dia3': [13, 14, 15, 16, 17, 18],
 'dia4': [19, 20, 21, 22, 23, 24],
 'dia5': [25, 2, 27, 28, 10, 30]}

True


In [10]:
def get_possible_solution(solution, schedule_matrix, do_local_search=False):
    solution_copy = copy.deepcopy(solution)

    if not validate_solution(solution, schedule_matrix):
        solution_copy = remove_duplicate_takes(solution)
        solution_copy = add_missing_takes(solution_copy, schedule_matrix)

    if do_local_search:
        return local_search(solution_copy, schedule_matrix)
    else:
        return solution_copy

# Ejemplo de uso
invalid_solution = {
    "dia1": [1, 1, 3, 4, 5, 6],
    "dia2": [7, 8, 3, 3, 11, 12],
    "dia3": [13, 1, 15, 16, 17, 18],
    "dia4": [19, 20, 1, 22, 23, 24],
    "dia5": [25, 5, 27, 28, 5, 30]
}

display(get_possible_solution(invalid_solution, tabla_np))



{'dia1': [1, 26, 3, 4, 5, 6],
 'dia2': [7, 8, 21, 10, 11, 12],
 'dia3': [13, 2, 15, 16, 17, 18],
 'dia4': [19, 20, 29, 22, 23, 24],
 'dia5': [25, 14, 27, 28, 9, 30]}

In [12]:
import copy
import random

def evolutionary_algorithm(initial_solution, population_size, num_epochs, schedule_matrix):
    # Generar población inicial
    population = []
    for _ in range(population_size):
        solution = copy.deepcopy(initial_solution)
        for day, takes in solution.items():
            num_takes_to_mutate = random.randint(1, 10)
            for _ in range(num_takes_to_mutate):
                index_to_mutate = random.randint(0, len(takes) - 1)
                takes[index_to_mutate] = random.randint(1, len(schedule_matrix))
        solution = get_possible_solution(solution, schedule_matrix)
        population.append(solution)

    # Por cada época
    for _ in range(num_epochs):
        # Generar hijos (soluciones mutadas)
        children = []
        for solution in population:
            child = copy.deepcopy(solution)
            day_to_mutate = "day" + str(random.randint(1, len(child)))
            if day_to_mutate in child:  # Verificar si el día existe en la solución
                take_to_mutate = random.randint(0, len(child[day_to_mutate]) - 1)
                child[day_to_mutate][take_to_mutate] = random.randint(1, len(schedule_matrix))
                child = get_possible_solution(child, schedule_matrix, True)  # Corregir posibles incongruencias generadas por la mutación
                children.append(child)

        # Unir padres e hijos y escoger los mejores
        population += children
        population.sort(key=lambda solution: objective_function(solution, schedule_matrix))
        population = population[:population_size]  # Conservar solo los mejores

    return population[0]  # Devolver el mejor individuo




In [46]:
def multi_start_evolutionary_algorithm(population_size, num_epochs, num_starts, schedule_matrix):
    best_solution = None
    best_cost = float('inf')
    countsesion = 0
    for i in range(num_starts):
        evolutionary_solution = evolutionary_algorithm(initial_solution, population_size, num_epochs, schedule_matrix)
        cost = objective_function(evolutionary_solution, schedule_matrix)
        if  countsesion <= 0:
          for day, takes in evolutionary_solution.items():
              actors = set()

              if int(day[-1]) <= 5:
                for take in takes:
                  actors.update(np.where(schedule_matrix[take - 1] == 1)[0] + 1)
                print(f"Tomas {day}: {takes}")

        if cost < best_cost:
            best_solution = evolutionary_solution
            best_cost = cost
        countsesion = countsesion + 1
    print(f"Mejor costo obtenido: {best_cost}")
    return best_solution

# Ejemplo de uso:
solucion_multiarranque = multi_start_evolutionary_algorithm(population_size=50, num_epochs=10, num_starts=5, schedule_matrix=tabla_np)


Tomas dia 1: [7, 2, 3, 4, 5, 11]
Tomas dia 2: [1, 12, 17, 19, 9, 6]
Tomas dia 3: [13, 20, 15, 26, 18, 10]
Tomas dia 4: [30, 8, 21, 22, 23, 24]
Tomas dia 5: [25, 16, 27, 28, 29, 14]
Mejor costo obtenido: 33


## Conclusiones



### Resultados:

- Se obtuvieron los siguientes resultados

        Tomas dia 1: [7, 2, 3, 4, 5, 11]
        Tomas dia 2: [1, 12, 17, 19, 9, 6]
        Tomas dia 3: [13, 20, 15, 26, 18, 10]
        Tomas dia 4: [30, 8, 21, 22, 23, 24]
        Tomas dia 5: [25, 16, 27, 28, 29, 14]
        Mejor costo obtenido: 33


 - El algoritmo voraz ha logrado agrupar las tomas en 5 días, con un máximo de 6 tomas por día.
El costo total de las sesiones de doblaje es de 33, lo que representa el mínimo posible considerando que cada actor cobra por día de trabajo.
La distribución de las tomas en los días parece ser equilibrada, sin que haya un día con una carga de trabajo excesiva.

 - Es importante recordar que el algoritmo voraz es una aproximación al problema de optimización y no siempre encuentra la solución óptima.
La calidad de la solución depende de la heuristica utilizada para seleccionar la toma con mayor cantidad de actores participantes.
En algunos casos, puede ser necesario explorar otras alternativas para encontrar una mejor solución.

- El resultado obtenido por el algoritmo voraz es una buena aproximación al problema de planificar las sesiones de doblaje. El costo total de las sesiones es el mínimo posible y la distribución de las tomas en los días parece ser equilibrada. Sin embargo, es importante tener en cuenta las limitaciones del algoritmo voraz y considerar otras alternativas para encontrar una mejor solución.




